In [1]:
from graphing_libraries import ThreeDPlotter, TimeSeriesPlotter, SubPlot3DPlotter
from data_generators import data_for_3d
import ipywidgets as widgets

import pandas as pd

from IPython.display import display
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt


In [2]:
df = data_for_3d()
scale_factor_df = pd.DataFrame({
    'scale': ["scale1", "scale2", "scale3", "scale4",""],
    'factor': [0.5, 1.0, 1.5, 2.0,0.0]
})
scale_factor_df = scale_factor_df.set_index("scale")

In [ ]:
# Enable interactive features
# %matplotlib widget

# plotter = ThreeDPlotter( dataframe=df)
# plotter.plot_3d_bar(columns_to_include=['Column1', 'Column2'], years_subset=[2010, 2011, 2013, 2014], growth_rate=0.05)
# plotter.plot_3d_line(columns_to_include=['Column1', 'Column2', 'Column3'], add_average=True, scale_factor=None, growth_rate=0.05,  show_original=True, years_subset=None)
# plotter.plot_3d_stem(columns_to_include=['Column1', 'Column2', 'Column3'], add_average=False, scale_factor=None, show_original=True, years_subset=None, growth_rate=0.05)

In [ ]:
%matplotlib widget
class Interactive3DPlot:
    def __init__(self, df, scale_factor_df):
        self.df = df
        self.scale_factor_df = scale_factor_df

        # Initialize widgets
        self.columns_widget = widgets.SelectMultiple(
            options=df.columns.tolist(),
            value=['Column1', 'Column2'],
            description='Columns to Include'
        )

        self.years_widget = widgets.SelectMultiple(
            options=df.index.unique().tolist(),
            value=[2010, 2011, 2013, 2014],
            description='Years Subset'
        )

        self.growth_rate_widget = widgets.FloatSlider(
            value=0.05,
            min=0.0,
            max=0.1,
            step=0.01,
            description='Growth Rate',
            style={'description_width': 'initial'}
        )

        self.scale_dataframe_checkbox = widgets.Checkbox(
            value=False,
            description='Scale DataFrame'
        )

        self.scale_factor_widget = widgets.Dropdown(
            options=scale_factor_df.index.tolist(),
            value="",  # Default scale factor index
            description='Scale Factor',
            style={'description_width': 'initial'}
        )

        self.interactive_plot = widgets.interactive(
            self.update_plot,
            columns_to_include=self.columns_widget,
            years_subset=self.years_widget,
            growth_rate=self.growth_rate_widget,
            scale_dataframe=self.scale_dataframe_checkbox,
            scale_factor=self.scale_factor_widget
        )

        # Attach the update_checkbox function to the checkbox's value attribute
        self.scale_dataframe_checkbox.observe(self.update_checkbox, names='value')

        # Display the interactive widget
        display(widgets.VBox([self.scale_dataframe_checkbox, self.interactive_plot]))

    def update_plot(self,columns_to_include, years_subset, growth_rate, scale_dataframe, scale_factor):
        # Scale all columns in the dataframe by the specified scale factor if the checkbox is selected
        factor = scale_factor_df.loc[scale_factor]['factor'] if scale_dataframe else 0.0
        scaled_df = df.copy()
        if (scale_dataframe) and (factor != 0.0):
            scaled_df = (scaled_df)*(1/factor)
            plotter = ThreeDPlotter(dataframe=scaled_df, kind = "scaled")
        else:
            plotter = ThreeDPlotter(dataframe=scaled_df)
        plotter.plot_3d_bar(columns_to_include=list(columns_to_include), years_subset=list(years_subset), growth_rate=growth_rate)

    def update_checkbox(self, change):
        self.scale_factor_widget.layout.visibility = 'visible' if change['new'] else 'hidden'

# Example usage:
interactive_plotter = Interactive3DPlot(df, scale_factor_df)


In [ ]:
%matplotlib widget
class InteractiveLinePlot:
    def __init__(self, df, scale_factor_df):
        self.df = df
        self.scale_factor_df = scale_factor_df
        self.columns_widget_line = widgets.SelectMultiple(
            options=self.df.columns.tolist(),
            value=('Column1', 'Column2', 'Column3'),
            description='Columns to Include'
        )
        self.years_widget_line = widgets.SelectMultiple(
            options=self.df.index.unique().tolist(),
            value=[2010, 2011, 2013, 2014],
            description='Years Subset'
        )
        self.add_average_widget = widgets.Checkbox(
            value=True,
            description='Add Average'
        )
        self.scale_factor_widget = widgets.FloatSlider(
            value=None,  # Set your desired default value
            min=0.0,
            max=2.0,
            step=0.1,
            description='Scale Factor',
            style={'description_width': 'initial'}
        )
        self.growth_rate_widget_line = widgets.FloatSlider(
            value=0.05,
            min=0.0,
            max=0.1,
            step=0.01,
            description='Growth Rate',
            style={'description_width': 'initial'}
        )
        self.show_original_widget = widgets.Checkbox(
            value=True,
            description='Show Original'
        )
        self.convert_series = widgets.Dropdown(
            options=self.scale_factor_df.index.tolist(),
            value="",  # Default value is an empty string
            description='Convert Series',
            style={'description_width': 'initial'}
        )
        self.convert_series_checkbox = widgets.Checkbox(
            value=False,
            description='Convert Series'
        )
        self.interactive_line_plot = widgets.interactive(
            self.update_line_plot,
            columns_to_include=self.columns_widget_line,
            years_subset=self.years_widget_line,
            add_average=self.add_average_widget,
            scale_factor=self.scale_factor_widget,
            growth_rate=self.growth_rate_widget_line,
            show_original=self.show_original_widget,
            convert_series=self.convert_series
        )
        display(widgets.VBox([self.convert_series_checkbox, self.convert_series, self.interactive_line_plot]))

        self.convert_series_checkbox.observe(self.update_convert_series_widget, names='value')

    def update_line_plot(self,columns_to_include,years_subset, add_average, scale_factor, growth_rate, show_original, convert_series):
        # Convert the series in df based on the selected convert_series value
    
        factor = scale_factor_df.loc[convert_series]['factor'] if convert_series else 0.0
        scaled_df = df.copy()
        if (convert_series) and (factor != 0.0):
            scaled_df = (scaled_df)*(1/factor)
            plotter = ThreeDPlotter(dataframe=scaled_df, kind = "Scaled")
        else:
            plotter = ThreeDPlotter(dataframe=scaled_df)
        plotter.plot_3d_line(columns_to_include=columns_to_include,
                             add_average=add_average,
                             scale_factor=scale_factor,
                             growth_rate=growth_rate,
                             show_original=show_original,
                             years_subset=list(years_subset))

    def update_convert_series_widget(self, change):
        self.convert_series.layout.visibility = 'visible' if change['new'] else 'hidden'

# Usage example:
interactive_plot = InteractiveLinePlot(df, scale_factor_df)


In [ ]:
%matplotlib widget
class Interactive3DStemPlotter:
    def __init__(self, df, scale_factor_df):
        self.df = df
        self.scale_factor_df = scale_factor_df
        self.create_widgets()
        self.create_callback()

    def create_widgets(self):
        # Define interactive widgets with default values
        self.columns_widget_stem = widgets.SelectMultiple(
            options=self.df.columns.tolist(),
            value=('Column1', 'Column2', 'Column3'),
            description='Columns to Include'
        )

        self.years_widget_stem = widgets.SelectMultiple(
            options=self.df.index.unique().tolist(),
            value=[2010, 2011, 2013, 2014],
            description='Years Subset'
        )

        self.add_average_widget_stem = widgets.Checkbox(
            value=False,
            description='Add Average'
        )

        self.scale_factor_widget_stem = widgets.FloatSlider(
            value=None,  # Set your desired default value
            min=0.0,
            max=2.0,
            step=0.1,
            description='Scale Factor',
            style={'description_width': 'initial'}
        )

        self.show_original_widget_stem = widgets.Checkbox(
            value=True,
            description='Show Original'
        )

        self.growth_rate_widget_stem = widgets.FloatSlider(
            value=0.05,
            min=0.0,
            max=0.1,
            step=0.01,
            description='Growth Rate',
            style={'description_width': 'initial'}
        )

        # Dropdown for converting series
        self.convert_series_stem = widgets.Dropdown(
            options=self.scale_factor_df.index.tolist(),
            value="",  # Default value is an empty string
            description='Convert Series',
            style={'description_width': 'initial'}
        )

        # Checkbox for whether to use convert_series
        self.convert_series_checkbox_stem = widgets.Checkbox(
            value=False,
            description='Convert Series'
        )

        # Create an interactive widget for the 3D stem plot
        self.interactive_stem_plot = widgets.interactive(
            self.update_stem_plot,
            columns_to_include=self.columns_widget_stem,
            years_subset=self.years_widget_stem,
            add_average=self.add_average_widget_stem,
            scale_factor=self.scale_factor_widget_stem,
            show_original=self.show_original_widget_stem,
            growth_rate=self.growth_rate_widget_stem,
            convert_series=self.convert_series_stem
        )

        # Display the interactive widget for the 3D stem plot
        display(widgets.VBox([self.convert_series_checkbox_stem, self.convert_series_stem, self.interactive_stem_plot]))

    def update_stem_plot(self, columns_to_include, years_subset, add_average, scale_factor, show_original, growth_rate, convert_series):
        # Convert the series in df based on the selected convert_series value
        factor = self.scale_factor_df.loc[convert_series]['factor'] if convert_series else 0.0
        scaled_df = self.df.copy()
        if (convert_series) and (factor != 0.0):
            scaled_df = (scaled_df) * (1 / factor)
            plotter = ThreeDPlotter(dataframe=scaled_df, kind="Scaled")
        else:
            plotter = ThreeDPlotter(dataframe=self.df)
        plotter.plot_3d_stem(columns_to_include=columns_to_include,
                             add_average=add_average,
                             scale_factor=scale_factor,
                             show_original=show_original,
                             years_subset=list(years_subset),
                             growth_rate=growth_rate)

    def create_callback(self):
        # Define a function to update the visibility of the convert_series widget based on the checkbox
        def update_convert_series_widget_stem(change):
            self.convert_series_stem.layout.visibility = 'visible' if change['new'] else 'hidden'

        # Attach the update_convert_series_widget function to the checkbox's value attribute
        self.convert_series_checkbox_stem.observe(update_convert_series_widget_stem, names='value')

# Example usage:
interactive_plotter = Interactive3DStemPlotter(df, scale_factor_df)


In [4]:
%matplotlib widget
class Interactive3DLinePlotter:
    def __init__(self, df):
        self.df = df
        self.create_widgets()
        self.create_callback()

    def create_widgets(self):
        # Define interactive widgets with default values
        self.columns_widget_line = widgets.SelectMultiple(
            options=self.df.columns.tolist(),
            value=self.df.columns.tolist(),
            description='Columns to Include'
        )

        self.years_widget_line = widgets.SelectMultiple(
            options=self.df.index.unique().tolist(),
            value=self.df.index.unique()[:3].tolist(),
            description='Years Subset'
        )

        # Create an interactive widget for the 3D line plot
        self.interactive_line_plot = widgets.interactive(
            self.update_line_plot,
            subset_columns=self.columns_widget_line,
            subset_years=self.years_widget_line
        )

        # Display the interactive widget for the 3D line plot
        display(self.interactive_line_plot)

    def update_line_plot(self, subset_columns, subset_years):
        plotter = SubPlot3DPlotter(self.df)
        plotter.plot_3d_line(subset_columns=list(subset_columns), subset_years=list(subset_years) if subset_years else None)

    def create_callback(self):
        # Define a function to update the options of the years widget based on the selected columns
        def update_years_options(change):
            selected_columns = change['new']
            all_years = self.df.index.unique().tolist()

            # If no columns are selected, display all years
            if not selected_columns:
                self.years_widget_line.options = all_years
                return

            # Display only the years that have data for the selected columns
            available_years = self.df[selected_columns].dropna().index.unique().tolist()
            self.years_widget_line.options = available_years

        # Attach the update_years_options function to the columns widget's value attribute
        self.columns_widget_line.observe(update_years_options, names='value')

# Example usage:
interactive_line_plotter = Interactive3DLinePlotter(df)


interactive(children=(SelectMultiple(description='Columns to Include', index=(0, 1, 2, 3, 4), options=('Column…